In [1]:
import requests as r
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import pickle

from threading import Thread
import time
from random import choice
import logging
logging.basicConfig(level = logging.INFO)
from threading import Thread

import pprint
pp = pprint.PrettyPrinter(indent=2)

from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

requests = r.Session()

retries = Retry(total=5,
                backoff_factor=2,
                status_forcelist=[ 500, 502, 503, 504 ],
                connect=10)
requests.mount("https://", HTTPAdapter(max_retries=retries))

In [55]:
v1 = json.load(open('../data_scraped/Restaurant_Scraping/scraped_restaurantreviews.json','r'))

In [11]:
def scrape(url, name, class_):
    """Scrapes `url` using requests and bs4 for a `class_` with the specific`name`,
        and returns the scraped section of interest, `scraped` as a result_set. 

    Uses randomized User-Agents as headers for the scraping to avoid being blocked by the site.
    """

    desktop_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
             'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
             'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
             'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14',
             'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
             'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
             'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
             'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
             'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
             'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0']

    def random_headers():
        return {'User-Agent': choice(desktop_agents),
                'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

    r = requests.get(url, timeout=10, headers=random_headers())
    soup = BeautifulSoup(r.text, 'html.parser')            
    scraped = soup.find_all(name, class_=class_)
    if scraped:
        return scraped
    raise Exception

In [56]:
#test
soup = scrape('http://www.burpple.com' + '/1001-of-arabia', 'div','venue-details__item-body')
soup[0].find('p').text

'\n23 Baghdad Street\nSingapore 199662\n'

In [57]:
# for s in soup:
#     x = s.find('p')
#     if x: print(x.text)

In [63]:
results_dict = v1
review_errors = set()
thread_list = []

def _get_info(i, restaurant, review_errors):
    try:
        url = 'http://www.burpple.com' + restaurant['Link']
        info_soup = scrape(url, 
                      name='div',
                      class_='venue-details__item-body')
        info = info_soup[0].find('p')
        if info: restaurant['Address'] = info.text.replace('\n',' ').strip()
        logging.info("Got address for %s", url)
    except Exception:
        logging.exception("error_in_thread for %s", url)
        review_errors.add(url)

for restaurant in results_dict.values():
    logging.info("scraped=%s", restaurant.get('Scraped'))
    if 'Scraped' in restaurant and restaurant['Scraped']: 
        logging.info('Skipping restaurant [%s] ', restaurant.keys())
        continue
    try:
        link = restaurant['Link']
        restaurantpage = f'http://www.burpple.com{link}'

        thread = Thread(target=_get_info, kwargs={"i":i, 
                                                 "restaurant": restaurant, 
                                                 "review_errors":review_errors
                                                }, daemon=True)
        thread.start()
        time.sleep(0.5)
        thread_list.append(thread)
        restaurant['Scraped'] = True
        
        # save after each scrape.
        with open('restaurant_info.json', 'w') as f:
            json.dump(results_dict, f, indent=4, ensure_ascii=False)    
                     
    except Exception as e:
        review_errors.add(link)
        logging.exception("error for restaurant %s",restaurant['Link'])
        
[t.join() for t in thread_list]
    

INFO:root:scraped=True
INFO:root:Skipping restaurant [dict_keys(['Link', 'Location', 'Latitude', 'Longitude', 'PriceRange', 'NumReviews', 'Reviews', 'DupeCount', 'Scraped', 'Categories', 'Tags', 'NumScraped'])] 
INFO:root:scraped=True
INFO:root:Skipping restaurant [dict_keys(['Link', 'Location', 'Latitude', 'Longitude', 'PriceRange', 'NumReviews', 'Reviews', 'DupeCount', 'Scraped', 'Categories', 'Tags', 'NumScraped'])] 
INFO:root:scraped=True
INFO:root:Skipping restaurant [dict_keys(['Link', 'Location', 'Latitude', 'Longitude', 'PriceRange', 'NumReviews', 'Reviews', 'DupeCount', 'Scraped', 'Categories', 'Tags', 'NumScraped'])] 
INFO:root:scraped=True
INFO:root:Skipping restaurant [dict_keys(['Link', 'Location', 'Latitude', 'Longitude', 'PriceRange', 'NumReviews', 'Reviews', 'DupeCount', 'Scraped', 'Categories', 'Tags', 'NumScraped'])] 
INFO:root:scraped=True
INFO:root:Skipping restaurant [dict_keys(['Link', 'Location', 'Latitude', 'Longitude', 'PriceRange', 'NumReviews', 'Reviews', 'Du

INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/tamoyasanuki-udon-1
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/rookery-1
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/hopii-singapore
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/omote
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/red-sparrow
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/kooks-creamery
INFO:root:Got address for http://www.burpple.com/good-bites
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/gogobentosg
INFO:root:Got address for http://www.burpple.com/toms-palette
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/godmama
INFO:root:Got address for http://www.burpple.com/skai
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:scraped=Fal

INFO:root:Got address for http://www.burpple.com/supply-demand-orchard-gateway
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/the-communal-place
INFO:root:Got address for http://www.burpple.com/oatberry-granola-kitchen
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/kei-kaisendon
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/pizza-giardino-chimes
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/five-izakaya-bar-ccp
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/ramen-champion
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/brine-singapore
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/kanpai-izakaya-bar
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/roots-mediterranean
INFO:root:Got address for http://www.burpple.com/spicy-rock
INFO:root

INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/wheat-somerset
INFO:root:Got address for http://www.burpple.com/a-noodle-story
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/empress
INFO:root:Got address for http://www.burpple.com/meatsmith-little-india
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/xiaolongkan-bedok
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/4u-bar-kitchen
INFO:root:Got address for http://www.burpple.com/siri-house
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/springleaf-jcube
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/the-clueless-goat
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/shin-minori-japanese-restaurant
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/t

INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/hock-hai-curry-chicken-noodle
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/casa-verde-3
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/safari-cafe
INFO:root:Got address for http://www.burpple.com/eat-3-bowls
INFO:root:Got address for http://www.burpple.com/tanuki-raw
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/canton-paradise-bishan-junction-8
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/nicoles-flower
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/mouth-restaurant-maxwell
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/power-coffeehouse
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/chef-chik
INFO:root:scraped=False
INFO:root:Got address for http://www.burpp

...

INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/beefupsingapore
INFO:root:Got address for http://www.burpple.com/hong-ji-mian-shi-jia-wanton-mee
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/good-year-local-hainanese-chicken-rice-ball-toa-payoh
INFO:root:Got address for http://www.burpple.com/aqua-luna-park-hotel-alexandra
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/baan-khun-nai
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/25-degrees-singapore
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/renku-lounge
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/kopifellas-1
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/lavo-singapore
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/lifestyle-cafe-bar
INFO:root:Got address for http://www.burpp

INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/soon-soon-huat-cafe
INFO:root:Got address for http://www.burpple.com/tan-yu-westgate
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/madeleines-original-portuguese-egg-tart-puff
INFO:root:Got address for http://www.burpple.com/nunsaram-korean-dessert-cafe
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/district-sushi
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/sungei-road-laksa
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/ikura-japanese-east-village
INFO:root:Got address for http://www.burpple.com/banchong
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/kream-beer
INFO:root:Got address for http://www.burpple.com/coastes
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/mei-heong-yuen-dessert

INFO:root:Got address for http://www.burpple.com/wildseed
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/konjiki-hototogisu
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/marche-movenpick-suntec-city
INFO:root:Got address for http://www.burpple.com/jin-biao-coffeeshop-potong-pasir
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/capitol-milk-bar
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/bizen-okayama-wagyu-steakhouse-1
INFO:root:Got address for http://www.burpple.com/frieda-restaurant
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/wildseed-cafe-at-the-alkaff-mansion
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/birds-of-a-feather-sg
INFO:root:Got address for http://www.burpple.com/the-big-cheese
INFO:root:scraped=False
INFO:root:Got address for http://www.bu

INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/kyoaji-japanese-dining
INFO:root:Got address for http://www.burpple.com/three-meals-a-day
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/bear-tea
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/taan
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/moose-yogurt
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/the-black-sheep-cafe
INFO:root:Got address for http://www.burpple.com/mings-prawn-noodle
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/ya-kun-kaya-toast-at-westmall
INFO:root:Got address for http://www.burpple.com/tanjong-beach-club
INFO:root:scraped=False
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/the-bakery-chef
INFO:root:scraped=False
INFO:root:Got address for http://www.burpple.com/tea-chapter
INFO:root:scraped=False
INFO:root:Go

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [82]:
count = 0
for k, v in results_dict.items():
    if 'Address' in v:
        count+=1
        address = v['Address'] 
#         if address.endswith('[0-9]{6}'):
#         if count < 10:
#         if re.search('Opening', address):
        if address == nan:
            print(k, address)
print(count, len(results_dict))

NameError: name 'nan' is not defined

In [88]:
results = pd.DataFrame.from_dict(results_dict, orient='index').reset_index()
results = results.drop(columns=['Scraped','DupeCount'])  # remove some redundant columns
results = results.rename({'index':'Title'}, axis=1)[['Title','Link','Location','Latitude','Longitude','NumReviews','PriceRange','Tags','Categories', 'Address']]
results.loc[results.Address.isna(), 'Link']

0              /babasan-by-uncle-kiisu
1          /enjoy-eating-house-and-bar
2                  /wu-pao-chun-bakery
3                     /thesaltedplumsg
4                          /rakki-bowl
5                              /elemen
6     /pasta-e-formaggio-marina-square
7                             /se7enth
8                    /goodtimeeatdrink
9            /elixir-boutique-roasters
10                            /lecoqsg
11         /breko-cafe-holland-village
12                      /cheeky_allday
13                               /yobo
14       /bundt-by-the-backyard-bakers
15                  /the-salted-plum-1
16          /shin-minori-katong-square
17             /social-place-singapore
18                          /fat-belly
19                /boufe-boutique-cafe
Name: Link, dtype: object

In [89]:
addresses = []
missing = results.loc[results.Address.isna(), 'Link'].tolist()
for missing_url in missing:
    url = 'http://www.burpple.com' + missing_url
    info_soup = scrape(url, 
                  name='div',
                  class_='venue-details__item-body')
    info = info_soup[0].find('p')
    if info: addresses.append(info.text.replace('\n',' ').strip())
addresses

['50 Circular Road Singapore 049405',
 '383 Jalan Besar Kam Leng Hotel Singapore 209001',
 '13 Stamford Road  #01-19/20 Capitol Singapore Singapore 178905',
 '10 Circular Road Singapore 049366',
 '72 Duxton Road Singapore 089531',
 '9 Raffles Boulevard #01-75A/76 Millenia Walk Singapore 039596',
 '6 Raffles Boulevard #02-100A/B Marina Square Singapore 039594',
 'Oakwood Premier OUE Singapore 6 Shenton Way, OUE Downtown 1, #07-01 Singapore 068809',
 '889 East Coast Road Singapore 459092',
 '10 Jalan Serene #01-03A Serene Centre Singapore 258748',
 '25 Church Street #01-04 Capital Square 2 Singapore 049482',
 '38 Lorong Mambong Singapore 277694',
 '74 Neil Road Singapore 088839',
 '182 Cecil Street #02-14 Frasers Tower Singapore 069547',
 '2 Havelock Road #01-22 Havelock II Singapore',
 '3 Temasek Boulevard #B1-122A Suntec City (West Wing) Singapore 038983',
 '88 East Coast Road #02-08 Katong Square Singapore 423371',
 '583 Orchard Road #01-22 Forum The Shopping Mall Singapore 238884',
 

In [91]:
results.loc[results.Address.isna(), 'Address'] = addresses

In [93]:
results.to_csv('../data_scraped/Restaurant_Scraping/restaurants_updated.csv')